In [ ]:
import boto3

eb = boto3.client('events')
kinesis = boto3.client('kinesis')

In [ ]:
import os
account_no = os.environ['ACCOUNT_NO']

In [ ]:
kinesis.create_stream(
    StreamName='destination', 
    ShardCount = 1)

In [ ]:
print(eb)
cbr = eb.create_event_bus(
    Name='greyhound'
)
print(cbr)

In [ ]:
# Create a role that lets the event bridge write to the destination stream
import json

kinesis_write_policy = {
    "Version": "2012-10-17",
    "Statement": [
        
        {
            "Sid": "WriteOutputKinesis",
            "Effect": "Allow",
            "Action": [
                "kinesis:DescribeStream",
                "kinesis:PutRecord",
                "kinesis:PutRecords"
            ],
            "Resource": [
                "arn:aws:kinesis:us-east-1:" + account_no + ":stream/destination"
            ]
        }
    ]
}

assume_role_policy = {
    "Statement":[{
        "Effect":"Allow",
        "Principal": {"Service":["events.amazonaws.com"]},
        "Action": ["sts:AssumeRole"]
    }]
}

iam = boto3.client('iam')

crr = iam.create_role(
    RoleName='sample-eb-role',
    Path='/service-role/',
    AssumeRolePolicyDocument=json.dumps(assume_role_policy)
)

print(crr)

prp = iam.put_role_policy(
    RoleName='sample-eb-role',
    PolicyName='EBPolicy',
    PolicyDocument=json.dumps(kinesis_write_policy)
)

print(prp)

In [ ]:
pattern = {
    "detail": {
        "type":["foo"]
    }
}

In [ ]:
eb.put_rule(
    Name='everything2stream',
    State='ENABLED', 
    RoleArn= "arn:aws:iam::" + account_no + ":role/service-role/sample-eb-role",
    #EventPattern=json.dumps(pattern),
    EventPattern='{"source":["yo.mamma"]}',
    EventBusName='greyhound'
)

In [ ]:
from datetime import datetime, timezone
import uuid

def timestamp():
    the_time = datetime.now(timezone.utc)
    return the_time.isoformat()

event1 = {
    "specversion":"1.0",
    "type":"newFoo",
    "source":"yo.mamma",
    "id":str(uuid.uuid4()),
    "time":timestamp(),
    "data":{"fooaddr":"foostuffval",
       "foolist": [1,2,3],
       "barobj": {
           "baraatr1":"yes",
           "barattr2":False,
           "barattr3":122.22
       }}
}

event2 = {
    "specversion":"1.0",
    "type":"foo",
    "source":"foo",
    "id":str(uuid.uuid4()),
    "time":timestamp(),
    "data":{"fooaddr":"foostuffval",
       "foolist": [1,2,3],
       "barobj": {
           "baraatr1":"yes",
           "barattr2":False,
           "barattr3":122.22
       }}
}

In [ ]:
# Try match
print(json.dumps(event2))

wrapped = {
  "version": "0",
  "id": "3e3c153a-8339-4e30-8c35-687ebef853fe",
  "detail-type": "EC2 Instance Launch Successful",
  "source": "aws.autoscaling",
  "account": "123456789012",
  "time": "2015-11-11T21:31:47Z",
  "region": "us-east-1",
  "resources": [
   ],
  "detail":event2
}
print(wrapped)
response = eb.test_event_pattern(
    EventPattern=json.dumps(pattern),
    Event=json.dumps(wrapped)
)
print(response)

wrapped['detail'] = event1
print(wrapped)

response = eb.test_event_pattern(
    EventPattern=json.dumps(pattern),
    Event=json.dumps(wrapped)
)

print(response)

In [ ]:
# Add a target for the rule
ptr = eb.put_targets(
    Rule='everything2stream',
    EventBusName='greyhound',
    Targets=[
        {
            'Id':'streamTarget',
            'Arn':'arn:aws:kinesis:us-east-1:' + account_no + ':stream/destination'
        }
    ]
)
print(ptr)

In [ ]:
# Put an event

wrapped = {
    "Detail":json.dumps(event2),
    "EventBusName":"grayhound",
    "Source":"yo.momma",
    "DetailType":"blah blah blah"
}

print(wrapped)

per = eb.put_events(
    Entries=[
        
            wrapped
        
    ]
)

print(per)

In [ ]:
# Did it show up in the stream?

ls = kinesis.list_shards(
    StreamName='destination'
)
print(ls)

shardId= ls['Shards'][0]['ShardId']
print(shardId)

gsi = kinesis.get_shard_iterator(
    StreamName='destination',
    ShardId=shardId,
    ShardIteratorType='TRIM_HORIZON'
)

print(gsi)

shardItor = gsi['ShardIterator']
print(shardItor)

recs = kinesis.get_records(
    ShardIterator=shardItor
)
print(recs)

### Clean Up

In [ ]:
eb.remove_targets(
    Rule='everything2stream',
    EventBusName='greyhound',
    Ids=[
        'streamTarget',
    ]
)

In [ ]:
eb.delete_rule(
    Name='everything2stream',
    EventBusName='greyhound'
)

In [ ]:
eb.delete_event_bus(
    Name='greyhound'
)

In [ ]:

kinesis.delete_stream(StreamName='destination')

In [ ]:
iam.delete_role_policy(
    RoleName='sample-eb-role',
    PolicyName='EBPolicy'
)

In [ ]:
iam.delete_role(
    RoleName='sample-eb-role'
)